In [70]:
from gGA.data import AtomicData, _keys
from ase.io import read
from gGA.data import OrbitalMapper
from dptb.data import OrbitalMapper as OrbitalMapper_dptb
import torch
from gGA.data import block_to_feature
from gGA.utils.make_kpoints import kmesh_sampling
from gGA.nn.hr2hk import GGAHR2HK
from dptb.nn import HR2HK

data = AtomicData.from_ase(read("/root/Hubbard/gGA/test/C_cube.vasp"), r_max=3.1)
basis = {"C":[3]}
idp_phy = OrbitalMapper(basis=basis, spin_deg=True)

In [71]:
R = torch.tensor([[ 0.3185, -0.4645, -0.8821,  1.1261, -0.8537, -1.0408],
        [ 0.1954,  0.2708,  1.3779, -0.3707, -0.9616, -0.3749],
        [-0.6813, -1.0173,  1.5940, -0.6498, -0.3480,  1.9673],
        [ 1.1107,  0.8750,  1.6304, -1.2158, -1.2708,  0.7721],
        [ 1.3959,  0.9850, -2.1727, -1.1793,  0.6760, -0.9001],
        [-0.0047, -0.2934, -1.0373,  0.4029,  1.1813,  0.5374]])

block = {
    "0_0_0_0_0": torch.zeros(3,3),
    "0_0_0_0_1": -torch.eye(3)*(1/6),
    "0_0_0_1_0": -torch.eye(3)*(1/6),
    "0_0_1_0_0": -torch.eye(3)*(1/6)
}

block_to_feature(data, idp_phy, block)

kpoints = torch.tensor(kmesh_sampling([5,5,5], False)).float()
data[_keys.KPOINT_KEY] = kpoints
data[_keys.R_MATRIX_KEY] = R.unsqueeze(0)
data = idp_phy(data)
data = AtomicData.to_AtomicDataDict(data)

In [72]:
hr2k = HR2HK(
    idp=OrbitalMapper_dptb(basis=idp_phy.basis),
)
ghr2k = GGAHR2HK(
        idp_phy=idp_phy,
        idp_aux=OrbitalMapper(basis=basis, spin_deg=False),
        naux=1
    )

In [73]:
data_org = hr2k(data.copy())
data_gga = ghr2k(data.copy())

In [83]:
(data_org["hamiltonian"][0], data_gga["hamiltonian"][0])

(tensor([[0.8090+0.j, 0.0000+0.j, 0.0000+0.j],
         [0.0000+0.j, 0.8090+0.j, 0.0000+0.j],
         [0.0000+0.j, 0.0000+0.j, 0.8090+0.j]]),
 tensor([[ 3.3780+0.0000e+00j, -0.3926-3.7253e-09j, -2.9390+0.0000e+00j,
          -2.0862+0.0000e+00j,  0.7567-1.4901e-08j, -0.0520+0.0000e+00j],
         [-0.3926+3.7253e-09j,  2.5992+0.0000e+00j,  1.3153-2.9802e-08j,
           3.3038+5.9605e-08j, -1.8848+0.0000e+00j, -2.4242-2.9802e-08j],
         [-2.9390+0.0000e+00j,  1.3153+2.9802e-08j,  6.8390+0.0000e+00j,
           2.9960+0.0000e+00j, -5.3849+0.0000e+00j, -0.7827+1.4901e-08j],
         [-2.0862+0.0000e+00j,  3.3038-5.9605e-08j,  2.9960+0.0000e+00j,
           6.7526+0.0000e+00j, -1.0115+1.4901e-08j, -2.8552+0.0000e+00j],
         [ 0.7567+1.4901e-08j, -1.8848+0.0000e+00j, -5.3849+0.0000e+00j,
          -1.0115-1.4901e-08j,  8.3307+0.0000e+00j,  1.4545-1.4901e-08j],
         [-0.0520+0.0000e+00j, -2.4242+2.9802e-08j, -0.7827-1.4901e-08j,
          -2.8552+0.0000e+00j,  1.4545+1.4901e-08

In [82]:
R @ (torch.eye(6) * 0.8090) @ R.T

tensor([[ 3.3780, -0.3926, -2.9389, -2.0862,  0.7567, -0.0520],
        [-0.3926,  2.5991,  1.3152,  3.3038, -1.8847, -2.4241],
        [-2.9389,  1.3152,  6.8389,  2.9959, -5.3847, -0.7827],
        [-2.0862,  3.3038,  2.9959,  6.7525, -1.0115, -2.8552],
        [ 0.7567, -1.8847, -5.3847, -1.0115,  8.3305,  1.4545],
        [-0.0520, -2.4241, -0.7827, -2.8552,  1.4545,  2.4340]])

In [61]:
N = 5
data_org["hamiltonian"][N], -(data["kpoint"][N] * torch.pi*2).cos().sum() / 3

(tensor([[0.4363+0.j, 0.0000+0.j, 0.0000+0.j],
         [0.0000+0.j, 0.4363+0.j, 0.0000+0.j],
         [0.0000+0.j, 0.0000+0.j, 0.4363+0.j]]),
 tensor(0.4363))